In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import mne
from scipy import signal
import meegkit
from meegkit import dss
from meegkit.utils import create_line_data, unfold, fold, rms, tscov
from meegkit.utils.denoise import demean
from meegkit.tspca import tsr

%matplotlib qt5

In [2]:
raw = mne.io.read_raw_fif('pre_filtered_data_raw.fif', preload=True)

Opening raw data file pre_filtered_data_raw.fif...
    Range : 25000 ... 1583999 =     50.000 ...  3167.998 secs
Ready.
Opening raw data file /home/ijekt/Documents/cs_cog_fat/pre_filtered_data_raw-1.fif...
    Range : 1584000 ... 2135999 =   3168.000 ...  4271.998 secs
Ready.
Reading 0 ... 2110999  =      0.000 ...  4221.998 secs...


In [7]:
print(raw.ch_names)

['EOG002', 'ECG003', 'ECG', 'BIO005', 'EEG001', 'EEG002', 'EEG003', 'EEG004', 'EEG005', 'EEG006', 'EEG007', 'EEG008', 'EEG009', 'EEG010', 'EEG011', 'EEG012', 'EEG013', 'EEG014', 'EEG015', 'EEG016', 'EEG017', 'EEG018', 'EEG019', 'EEG020', 'EEG021', 'EEG022', 'EEG023', 'EEG024', 'EEG025', 'EEG026', 'EEG027', 'EEG028', 'EEG029', 'EEG030', 'MEG0111', 'MEG0112', 'MEG0113', 'MEG0121', 'MEG0122', 'MEG0123', 'MEG0131', 'MEG0132', 'MEG0133', 'MEG0141', 'MEG0142', 'MEG0143', 'MEG0211', 'MEG0212', 'MEG0213', 'MEG0221', 'MEG0222', 'MEG0223', 'MEG0231', 'MEG0232', 'MEG0233', 'MEG0241', 'MEG0242', 'MEG0243', 'MEG0311', 'MEG0312', 'MEG0313', 'MEG0321', 'MEG0322', 'MEG0323', 'MEG0331', 'MEG0332', 'MEG0333', 'MEG0341', 'MEG0342', 'MEG0343', 'MEG0411', 'MEG0412', 'MEG0413', 'MEG0421', 'MEG0422', 'MEG0423', 'MEG0431', 'MEG0432', 'MEG0433', 'MEG0441', 'MEG0442', 'MEG0443', 'MEG0511', 'MEG0512', 'MEG0513', 'MEG0521', 'MEG0522', 'MEG0523', 'MEG0531', 'MEG0532', 'MEG0533', 'MEG0541', 'MEG0542', 'MEG0543', 'M

In [3]:
# Define a minimum duration for events, e.g., 10 milliseconds
min_duration = 10 / 1000  # Convert milliseconds to seconds
min_duration_samples = min_duration * raw.info['sfreq']  # Convert to samples

In [4]:
events = mne.find_events(raw, initial_event=True, stim_channel=["STI101", "STI102"], min_duration=min_duration)

8027 events found on stim channel STI101
Event IDs: [    1     2     7     9    32    33    64    65    66    67    69    70
    71    72 32696 32697 32698 32699 32701 32702 32703 32704 32717 32719
 32759 32760 32761 32762 32763 32764 32765 32766 32767 32768]
2135 events found on stim channel STI102
Event IDs: [  3   4   7   8   9 254]


/tmp/ipykernel_21892/2991666.py:1: RuntimeWarning: Trigger channel contains negative values, using absolute value. If data were acquired on a Neuromag system with STI016 active, consider using uint_cast=True to work around an acquisition bug
  events = mne.find_events(raw, initial_event=True, stim_channel=["STI101", "STI102"], min_duration=min_duration)


In [3]:
events = mne.make_fixed_length_events(raw, start=0, stop=raw.times[-1], duration=1.0)
tmin, tmax = 0, 1  # Start and end time around event (in seconds)
baseline = None  # or (None, 0) if you want to apply baseline correction

# Create epochs
epochs = mne.Epochs(raw, events, tmin=tmin, tmax=tmax, baseline=baseline, preload=True)

Not setting metadata
4221 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 4221 events and 501 original time points ...
0 bad epochs dropped


In [4]:
epochs.plot()

Using qt as 2D backend.


Dropped 86 epochs: 131, 132, 133, 134, 135, 138, 139, 140, 141, 260, 261, 262, 263, 264, 323, 324, 359, 382, 383, 384, 385, 386, 406, 407, 416, 417, 808, 809, 810, 811, 812, 813, 814, 815, 816, 817, 818, 819, 820, 821, 822, 823, 824, 825, 826, 827, 828, 903, 904, 923, 924, 925, 926, 927, 928, 929, 930, 962, 963, 1013, 1014, 1321, 1322, 1323, 1324, 1496, 1497, 1498, 1499, 1760, 1761, 1762, 3822, 3823, 3824, 3825, 3826, 3866, 3871, 3872, 3887, 3888, 3889, 4211, 4212, 4213
The following epochs were marked as bad and are dropped:
[131, 132, 133, 134, 135, 138, 139, 140, 141, 260, 261, 262, 263, 264, 323, 324, 359, 382, 383, 384, 385, 386, 406, 407, 416, 417, 808, 809, 810, 811, 812, 813, 814, 815, 816, 817, 818, 819, 820, 821, 822, 823, 824, 825, 826, 827, 828, 903, 904, 923, 924, 925, 926, 927, 928, 929, 930, 962, 963, 1013, 1014, 1321, 1322, 1323, 1324, 1496, 1497, 1498, 1499, 1760, 1761, 1762, 3822, 3823, 3824, 3825, 3826, 3866, 3871, 3872, 3887, 3888, 3889, 4211, 4212, 4213]
Channels m

In [6]:
epochs.save('prefiltered_data_ca_dropped_epo.fif', overwrite=True)

Splitting into 2 parts


In [7]:
data = epochs.get_data()

/tmp/ipykernel_22189/2927246647.py:1: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  data = epochs.get_data()


In [9]:
print(data.shape)

(4135, 343, 501)


In [3]:
# Select only MEG channels
meg_channels = mne.pick_types(raw.info, meg=True)

# Select the EOG channel(s)
#eog_channels = mne.pick_types(raw.info, eog=True)

In [4]:
# Extract the data
meg_data = raw.get_data()[meg_channels]


In [5]:
#eog_data = raw.get_data()[eog_channels]

In [6]:
print(meg_data.shape)
#print(eog_data.shape)

(306, 2111000)


# remove powerline

In [8]:
# Set the parameters
fline = 50  # Power line frequency in Hertz
sfreq = raw.info['sfreq']  # Sampling frequency in Hertz
print(sfreq)

500.0


In [ ]:
# Apply DSS to remove power line artifacts on meg data
dss_meg, iterations = meegkit.dss.dss_line_iter(dss_pl_meg, fline, sfreq, nfft=400)
print(f"Removed {iterations} components")

In [9]:
np.save('dss_pl_meg.npy', dss_meg)

In [3]:
dss_pl_meg = np.load('dss_pl_meg.npy')

In [4]:
print(dss_pl_meg.shape)

(2111000, 306)


In [17]:
#print(eog_data.shape)

(1, 2111000)


In [18]:
#eog_data = eog_data.T

In [5]:
x = dss_pl_meg
sf = raw.info['sfreq']
x = demean(x)

In [6]:
# Remove eyeblink components
w = np.abs(x) > 4 * np.median(np.abs(x))
w = np.mean(w, axis=1)
c0 = np.cov(x, rowvar=False)
c1 = np.cov(np.multiply(x, w[:, np.newaxis]), rowvar=False)

In [7]:
print(w.shape)
print(c0.shape)
print(c1.shape)

(2111000,)
(306, 306)
(306, 306)


In [8]:
#apply dss
todss, fromdss, pwr0, pwr1,  = meegkit.dss.dss0(c0, c1)

In [9]:
print(pwr0.shape)
print(pwr1.shape)

(76,)
(76,)


In [10]:
# 4. Plotting
plt.figure(4)
plt.subplot(121)
plt.plot(pwr1 / pwr0, '.-')
plt.ylabel('score')
plt.xlabel('components')
plt.title('DSS eyeblink')

# Applying the DSS matrix to the data
z = np.matmul(x, todss)

In [11]:
# 1. Set number of components to remove
NREMOVE3 = 2

# 2. Perform Time Series Regression (TSR)
# This removes the contribution of the first NREMOVE3 components in z from x
x_cleaned = meegkit.tspca.tsr(x, z[:, :NREMOVE3])
cleaned_data = x_cleaned[0]

# 3. Plotting
plt.figure(5)
plt.clf()
plt.plot(cleaned_data)
plt.title('after eyeblink removal')
plt.draw()  # Equivalent to MATLAB's drawnow

In [ ]:
# Number of components to keep
NKEEP = 7

# Select the best components and project back to sensor space
projection_matrix = np.dot(todss[:, :NKEEP], fromdss[:NKEEP, :])
yy = np.dot(x, projection_matrix)

In [53]:
print(yy[:5])

[[-4.69034945e-18 -1.19738118e-16 -2.28340185e-17 ...  1.84440618e-18
  -7.51516969e-17 -6.04484052e-17]
 [ 1.64442318e-13  1.20822041e-12  1.17501873e-12 ... -2.93920798e-14
  -6.78132698e-13  2.28915155e-13]
 [ 9.76219957e-14  1.16983486e-12  4.13772913e-13 ... -6.43970745e-15
  -7.17776876e-13 -2.84579748e-13]
 [ 9.49949977e-14  1.96523967e-12  4.86528623e-13 ... -8.84341947e-15
  -1.17589874e-12 -1.69186303e-13]
 [ 1.50240101e-13  2.41068533e-12  8.71401864e-13 ... -2.09595867e-14
  -1.45620407e-12  1.64071666e-14]]


In [54]:
y = yy.T

In [55]:
n_channels = y.shape[0]

# Generate dummy channel names
channel_names = ['MEG%03d' % i for i in range(1, n_channels + 1)]

# Create a list of channel types (all 'meg' in this case)
channel_types = ['mag'] * n_channels

# Create MNE Info object
info = mne.create_info(ch_names=channel_names, sfreq=1000, ch_types=channel_types)  # Replace 1000 with your actual sampling rate

# Transpose 'y' to have shape [n_times, n_channels] and create RawArray object
dss_raw = mne.io.RawArray(y, info)

Creating RawArray with float64 data, n_channels=306, n_times=2111000
    Range : 0 ... 2110999 =      0.000 ...  2110.999 secs
Ready.


In [56]:
dss_raw.plot(duration=5, n_channels=20, scalings='auto')

Waiting for Loading-Thread to finish... (max. 10 sec)
Channels marked as bad:
none


Traceback (most recent call last):
  File "/home/ijekt/anaconda3/envs/neuroimaging/lib/python3.9/site-packages/mne_qt_browser/_pg_figure.py", line 2585, in run
    if self.mne.is_epochs:
AttributeError: 'LoadThread' object has no attribute 'mne'
